In [1]:
import requests
import pandas as pd
from config import CENSUS_API_KEY

In [3]:
def get_census_data(api_url, table_name, state_fips, county_fips, block_group):
    api_key = CENSUS_API_KEY
    url = api_url

    params = {
        "get": f"NAME,group({table_name})",
        # "for": f"county:{county_fips}",
        # "in": f"state:{state_fips}",
        "for": f"block group:{block_group}",
        "in": f"state:{state_fips} county:{county_fips}",
        "key": api_key,
    }

    response = requests.get(url, params=params)
    print(response.url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        return df
    else:
        print("Failed to retrieve data:", response.status_code)


In [4]:
# tables = 'B08134'
tables = 'B08302'
api_url = "https://api.census.gov/data/2022/acs/acs5"

# tables = 'P034'
# api_url = "https://api.census.gov/data/2000/dec/sf3"

df = get_census_data(api_url, tables, "47", "065", "*")

https://api.census.gov/data/2022/acs/acs5?get=NAME%2Cgroup%28B08302%29&for=block+group%3A%2A&in=state%3A47+county%3A065&key=d5bd87e9fe4db19d2c5c5150ec106748a006a2dd


In [64]:
new_columns = {
    'P034001': 'Total',
    'P034002': 'Did not work at home',
    'P034003': '12:00 a.m. to 4:59 a.m.',
    'P034004': '5:00 a.m. to 5:29 a.m.',
    'P034005': '5:30 a.m. to 5:59 a.m.',
    'P034006': '6:00 a.m. to 6:29 a.m.',
    'P034007': '6:30 a.m. to 6:59 a.m.',
    'P034008': '7:00 a.m. to 7:29 a.m.',
    'P034009': '7:30 a.m. to 7:59 a.m.',
    'P034010': '8:00 a.m. to 8:29 a.m.',
    'P034011': '8:30 a.m. to 8:59 a.m.',
    'P034012': '9:00 a.m. to 9:59 a.m.',
    'P034013': '10:00 a.m. to 10:59 a.m.',
    'P034014': '11:00 a.m. to 11:59 a.m.',
    'P034015': '12:00 p.m. to 3:59 p.m.',
    'P034016': '4:00 p.m. to 11:59 p.m.',
    'P034017': 'Worked at home'
}
df = df.rename(new_columns, axis=1)

In [5]:
columns_to_be_renamed = {
    'B08302_001E':'total_estimate', 'B08302_001M': 'total_margin', 
    'B08302_002E':'12am_to_4:59am_estimate', 'B08302_002M':'12am_to_4:59am_margin', 
    'B08302_003E':'5am_to_5:29am_estimate', 'B08302_003M':'5am_to_5:29am_margin', 
    'B08302_004E':'5:30am_to_5:59am_estimate', 'B08302_004M':'5:30am_to_5:59am_margin', 
    'B08302_005E':'6am_to_6:29am_estimate', 'B08302_005M':'6am_to_6:29am_margin', 
    'B08302_006E':'6:30am_to_6:59am_estimate', 'B08302_006M':'6:30am_to_6:59am_margin', 
    'B08302_007E':'7am_to_7:29am_estimate', 'B08302_007M':'7am_to_7:29am_margin', 
    'B08302_008E':'7:30am_to_7:59am_estimate', 'B08302_008M':'7:30am_to_7:59am_margin', 
    'B08302_009E':'8am_to_8:29am_estimate', 'B08302_009M':'8am_to_8:29am_margin', 
    'B08302_010E':'8:30am_to_8:59am_estimate', 'B08302_010M':'8:30am_to_8:59am_margin', 
    'B08302_011E':'9am_to_9:59am_estimate', 'B08302_011M':'9am_to_9:59am_margin', 
    'B08302_012E':'10am_to_10:59am_estimate', 'B08302_012M':'10am_to_10:59am_margin', 
    'B08302_013E':'11am_to_11:59am_estimate', 'B08302_013M':'11am_to_11:59am_margin', 
    'B08302_014E':'12pm_to_3:59pm_estimate', 'B08302_014M':'12pm_to_3:59pm_margin', 
    'B08302_015E':'4pm_to_11:59pm_estimate', 'B08302_015M':'4pm_to_11:59pm_margin', 
}

In [43]:
df_renamed = df[list(columns_to_be_renamed.keys()) + ['GEO_ID', 'state', 'county', 'tract', 'block group']].rename(columns_to_be_renamed, axis=1)
df_renamed['GEO_ID'] = df_renamed['GEO_ID'].apply(lambda x: x.split('US')[1].lstrip('0'))
df_renamed['total_estimate'] = df_renamed['total_estimate'].astype(int)

In [44]:
for column in df_renamed.columns:
    if 'estimate' in column or 'margin' in column:
        df_renamed[column] = df_renamed[column].astype('Int64')

In [48]:
df_renamed.head(1)

,total_estimate,total_margin,12am_to_4:59am_estimate,12am_to_4:59am_margin,5am_to_5:29am_estimate,5am_to_5:29am_margin,5:30am_to_5:59am_estimate,5:30am_to_5:59am_margin,6am_to_6:29am_estimate,6am_to_6:29am_margin,...,11am_to_11:59am_margin,12pm_to_3:59pm_estimate,12pm_to_3:59pm_margin,4pm_to_11:59pm_estimate,4pm_to_11:59pm_margin,GEO_ID,state,county,tract,block group
0,368,166,0,14,0,14,8,16,33,55,...,51,29,36,38,43,470650004001,47,065,000400,1


In [71]:
temp = df.rename(columns_to_be_renamed, axis=1).transpose().reset_index().rename({'index': 'time_of_departure', 0: 'counts'}, axis=1)
temp = temp.dropna(subset=['counts'])

In [72]:
temp

,time_of_departure,counts,1,2,3,4,5,6,7,8,...,268,269,270,271,272,273,274,275,276,277
0,NAME,Block Group 1; Census Tract 4; Hamilton County...,Block Group 2; Census Tract 4; Hamilton County...,Block Group 3; Census Tract 4; Hamilton County...,Block Group 1; Census Tract 6; Hamilton County...,Block Group 2; Census Tract 6; Hamilton County...,Block Group 3; Census Tract 6; Hamilton County...,Block Group 1; Census Tract 7; Hamilton County...,Block Group 2; Census Tract 7; Hamilton County...,Block Group 3; Census Tract 7; Hamilton County...,...,Block Group 2; Census Tract 123; Hamilton Coun...,Block Group 3; Census Tract 123; Hamilton Coun...,Block Group 1; Census Tract 124; Hamilton Coun...,Block Group 2; Census Tract 124; Hamilton Coun...,Block Group 3; Census Tract 124; Hamilton Coun...,Block Group 4; Census Tract 124; Hamilton Coun...,Block Group 1; Census Tract 9801; Hamilton Cou...,Block Group 1; Census Tract 9802; Hamilton Cou...,Block Group 2; Census Tract 9802; Hamilton Cou...,Block Group 3; Census Tract 9802; Hamilton Cou...
1,total_estimate,368,0,682,394,548,418,483,525,670,...,389,756,451,708,563,1244,0,0,0,0
3,total_margin,166,14,287,110,127,167,149,235,217,...,190,201,156,424,168,411,14,14,14,14
5,12am_to_4:59am_estimate,0,0,182,0,0,36,0,16,0,...,48,39,0,14,7,0,0,0,0,0
7,12am_to_4:59am_margin,14,14,205,14,14,56,14,25,14,...,57,45,14,22,11,14,14,14,14,14
9,5am_to_5:29am_estimate,0,0,26,24,10,6,0,0,0,...,11,0,9,45,12,26,0,0,0,0
11,5am_to_5:29am_margin,14,14,41,25,15,10,14,14,14,...,18,14,14,63,18,38,14,14,14,14
13,5:30am_to_5:59am_estimate,8,0,74,10,0,0,0,0,8,...,28,135,0,0,53,37,0,0,0,0
15,5:30am_to_5:59am_margin,16,14,83,14,14,14,14,14,13,...,42,141,14,14,61,51,14,14,14,14
17,6am_to_6:29am_estimate,33,0,0,19,50,0,3,0,54,...,32,39,0,0,28,16,0,0,0,0


In [16]:
def extract_times(time_str):
    if 'to_' in time_str:
        start_time, end_time = time_str.split('_to_')
        end_time = end_time.split('_')[0]
        # Handle time formatting
        start_time = start_time.replace('am', ' AM').replace('pm', ' PM')
        end_time = end_time.replace('am', ' AM').replace('pm', ' PM')
        return start_time, end_time
    return None, None

temp['start_time'], temp['end_time'] = zip(*temp['time_of_departure'].apply(extract_times))
# temp['estimate'] = temp[['time_of_departure', 'counts']].apply(lambda row: row['counts'] if row['time_of_departure'].split('_')[-1] == 'estimate' else None, axis=1)
# temp['margin'] = temp[['time_of_departure', 'counts']].apply(lambda row: row['counts'] if row['time_of_departure'].split('_')[-1] == 'margin' else None, axis=1)

In [17]:
df_estimate = temp[temp['time_of_departure'].str.contains('estimate')].copy()
df_margin = temp[temp['time_of_departure'].str.contains('margin')].copy()

# Rename the 'counts' column to 'estimate' in df_estimate and 'margin' in df_margin
df_estimate.rename(columns={'counts': 'estimate'}, inplace=True)
df_margin.rename(columns={'counts': 'margin'}, inplace=True)

# Drop unnecessary columns in each DataFrame
df_estimate.drop(['time_of_departure'], axis=1, inplace=True)
df_margin.drop(['time_of_departure'], axis=1, inplace=True)

# Merge the two DataFrames on 'start_time' and 'end_time'
df_combined = pd.merge(df_estimate, df_margin, on=['start_time', 'end_time'], how='outer')

# Fill NaNs with 'None' if needed
df_combined.fillna('None', inplace=True)
df_combined['start_time'].iloc[0] = '12 AM'
df_combined['end_time'].iloc[0] = '11:59 PM'

In [19]:
def convert_time(time_str):
    if time_str == 'None' or pd.isna(time_str):
        return None  # Or return a default value like '00:00:00'
    else:
        try:
            # Try to convert with full minute specification
            return pd.to_datetime(time_str, format='%I:%M %p', errors='raise').strftime('%H:%M:%S')
        except ValueError:
            # Fallback for hour only format
            try:
                return pd.to_datetime(time_str, format='%I %p', errors='raise').strftime('%H:%M:%S')
            except ValueError:
                # If neither format works, return None or an error message
                return None

# Apply the conversion function to start_time and end_time columns
df_combined['start_time'] = df_combined['start_time'].apply(convert_time)
df_combined['end_time'] = df_combined['end_time'].apply(convert_time)

In [20]:
df_combined

,estimate,start_time,end_time,margin
0,159014,00:00:00,23:59:00,2482
1,6042,00:00:00,04:59:00,706
2,4329,05:00:00,05:29:00,667
3,6107,05:30:00,05:59:00,512
4,12110,06:00:00,06:29:00,1025
5,15279,06:30:00,06:59:00,1169
6,22027,07:00:00,07:29:00,1241
7,22766,07:30:00,07:59:00,1163
8,17589,08:00:00,08:29:00,1044
9,9992,08:30:00,08:59:00,835


In [69]:
columns_to_be_renamed = {'B08134_001E':'total_estimate', 'B08134_001M': 'total_margin', 
                         'B08134_002E':'less_than_10_mins_estimate', 'B08134_002M':'less_than_10_mins_margin', 
                         'B08134_003E':'10_to_14_mins_estimate', 'B08134_003M':'10_to_14_mins_margin',
                         'B08134_004E':'15_to_19_mins_estimate', 'B08134_004M':'15_to_19_mins_margin',
                         'B08134_005E':'20_to_24_mins_estimate', 'B08134_005M':'20_to_24_mins_margin',
                         'B08134_006E':'25_to_29_mins_estimate', 'B08134_006M':'25_to_29_mins_margin',
                         'B08134_007E':'30_to_34_mins_estimate', 'B08134_007M':'30_to_34_mins_margin',
                         'B08134_008E':'35_to_44_mins_estimate', 'B08134_008M':'35_to_44_mins_margin',
                         'B08134_009E':'44_to_59_mins_estimate', 'B08134_009M':'44_to_59_mins_margin',
                         'B08134_010E':'60_or_more_mins_estimate', 'B08134_010M':'60_or_more_mins_margin',
                         'B08134_011E':'personal_vehicle_estimate', 'B08134_011M':'personal_vehicle_margin',
                         'B08134_021E':'drove_alone_estimate', 'B08134_021M':'drove_alone_margin',
                         'B08134_031E':'carpooled_estimate', 'B08134_031M':'carpooled_margin',
                         'B08134_041E':'two_person_carpool_estimate', 'B08134_041M':'two_person_carpool_margin',
                         'B08134_051E':'three_or_more_person_carpool_estimate', 'B08134_051M':'three_or_more_person_carpool_margin',
                         'B08134_061E':'public_transit_estimate', 'B08134_061M':'public_transit_margin',
                         'B08134_071E':'bus_estimate', 'B08134_071M':'bus_margin',
                         'B08134_081E':'subway_estimate', 'B08134_081M':'subway_margin',
                         'B08134_091E':'long_dist_train_estimate', 'B08134_091M':'long_dist_train_margin',
                         'B08134_101E':'walked_estimate', 'B08134_101M':'walked_margin',
                         'B08134_111E':'taxi_motorbike_bicycle_others_stimate', 'B08134_111M':'taxi_motorbike_bicycle_others_margin',
                         } 

In [6]:
df.rename(columns_to_be_renamed, axis=1)

,NAME,total_estimate,B08134_001EA,total_margin,B08134_001MA,less_than_10_mins_estimate,B08134_002EA,less_than_10_mins_margin,B08134_002MA,10_to_14_mins_estimate,...,B08134_119M,B08134_119MA,B08134_120E,B08134_120EA,B08134_120M,B08134_120MA,GEO_ID,NAME,state,county
0,"Hamilton County, Tennessee",159014,None,2482,None,18543,None,1150,None,23120,...,83,None,160,None,85,None,0500000US47065,"Hamilton County, Tennessee",47,065
